In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
import transformers
from torch import bfloat16
import time
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

import os

from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline

In [2]:
bnb_config_4 = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype=bfloat16
)

# model_id = "NousResearch/Llama-2-7b-chat-hf"
model_id = "Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir='models/', quantization_config=bnb_config_4,  device_map={"": 0}) # cache_dir  


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    pad_token_id = 50256,
    repetition_penalty=1.1
)

local_llm = HuggingFacePipeline(pipeline=pipe) #model_kwargs={"temperature":1e-10}

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
prompt_template = """Please provide honest and straightforward answers to the following questions. Avoid providing misleading or inaccurate information. Your responses are expected to be truthful and directly address the questions asked.

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["question"]
)



In [4]:
chain = LLMChain(llm=local_llm, prompt=PROMPT)

In [5]:
%%time

# full example
query = "Write a short peom about rain"
llm_response = chain.run(query)

print(prompt_template.format(question=query) + llm_response)
print("\n\n")

Please provide honest and straightforward answers to the following questions. Avoid providing misleading or inaccurate information. Your responses are expected to be truthful and directly address the questions asked.

Question: Write a short peom about rain
Answer: Sure, here is a short poem about rain:
Rain, oh rain, how you bring us cheer
Washing away our fears and drying our tears
With every drop, a new beginning starts
A chance for growth, and a brand new start
The smell of wet earth fills the air
As we dance in the puddles with joy beyond compare
Rain, oh rain, you're a gift from above
Bringing life to the world, and all its love.



CPU times: user 3.71 s, sys: 184 ms, total: 3.9 s
Wall time: 3.9 s


In [7]:
%%time

# full example
query = "Write a small story about kids"
llm_response = chain.run(query)

print(prompt_template.format(question=query) + llm_response)
print("\n\n")

Please provide honest and straightforward answers to the following questions. Avoid providing misleading or inaccurate information. Your responses are expected to be truthful and directly address the questions asked.

Question: Write a small story about kids
Answer: Sure! Here is a small story about kids:
Once upon a time, there were three kids named Jack, Jill, and Tom. They lived in a small town surrounded by rolling hills and green forests. Every day, they would play together in the woods, exploring new paths and discovering hidden streams. One sunny afternoon, while wandering deeper into the forest than ever before, they stumbled upon an old, mysterious-looking door hidden behind a thick cluster of bushes. Curious, they pushed the door open and found themselves in a magical garden filled with talking animals, colorful flowers, and a sparkling fountain. The kids played with the animals, picked wildflowers, and drank from the fountain until the sun began to set. As they left the gard